In [10]:
#### This is a cell to hide code snippets from displaying
#### This must be at first cell!

from IPython.display import HTML

hide_me = ''
HTML('''<script>
code_show=true; 
function code_toggle() {
  if (code_show) {
    $('div.input').each(function(id) {
      el = $(this).find('.cm-variable:first');
      if (id == 0 || el.text() == 'hide_me') {
        $(this).hide();
      }
    });
    $('div.output_prompt').css('opacity', 0);
  } else {
    $('div.input').each(function(id) {
      $(this).show();
    });
    $('div.output_prompt').css('opacity', 1);
  }
  code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input style="opacity:0" type="Eingabe" value="Click here to toggle on/off the raw code."></form>''')

In [4]:
hide_me

from metakernel import register_ipython_magics
register_ipython_magics()

import ipywidgets as widgets
import sys
from IPython.display import display
from IPython.display import clear_output

from pythonds.basic import Queue


def create_multipleChoice_widget(description, options, correct_answer):
    if correct_answer not in options:
        options.append(correct_answer)
    
    correct_answer_index = options.index(correct_answer)
    
    radio_options = [(words, i) for i, words in enumerate(options)]
    alternativ = widgets.RadioButtons(
        options = radio_options,
        description = '',
        disabled = False
    )
    
    description_out = widgets.Output()
    with description_out:
        print(description)
        
    feedback_out = widgets.Output()

    def check_selection(b):
        a = int(alternativ.value)
        if a==correct_answer_index:
            s = '\x1b[6;30;42m' + "Richtig." + '\x1b[0m' +"\n" #green color
        else:
            s = '\x1b[5;30;41m' + "Fail. " + '\x1b[0m' +"\n" #red color
        with feedback_out:
            clear_output()
            print(s)
        return
    
    check = widgets.Button(description="submit")
    check.on_click(check_selection)
    
    
    return widgets.VBox([description_out, alternativ, check, feedback_out])


frage1 ='Was ist das Ein/Ausgabeprinzip beim ADT Queue?'
frage2 ="""Was ist der Inhalt der Queue bei folgenden Operationen?
q = Queue()
q.enqueue(hello)
m.enqueue(dog)
m.enqueue(3)
m.dequeue()
"""

Q1 = create_multipleChoice_widget(frage1,['First-In, First-Out (FIFO)','Last-In, First-Out (LIFO)','Last-Out, First-In (LOFI)'],'First-In, First-Out (FIFO)')
Q2 = create_multipleChoice_widget(frage2, ['hello,dog','dog,3','hello,3','hello,dog,3'],'dog,3')


[<img src="Bilder/MIREVIBanner.jpg">](/../../FMA_start_here.ipynb)

# Abstrakte Datenstrukturen, Teil 3
Teil 3 der interaktiven FMA-Lerneinheit zu abstrakten Datenstrukturen

## Ziele 
- die abstrakten Datentypen Stack, Queue, Deque und List verstehen.
- die ADTs Stack, Queue und Deque unter Verwendung von Python-Listen implementieren können.
- die Performanz der Implementierungen von grundlegenden linearen Datenstrukturen zu verstehen.
- Verstehen der Präfix-, Infix- und Postfix-Ausdrucksformate.
- Verwendung von Stacks zur Auswertung von Postfix-Ausdrücken.
- Verwendung von Stacks zur Konvertierung von Ausdrücken von Infix- in Postfix-Ausdrücke.
- **Verwendung von Warteschlangen für grundlegende Timing-Simulationen.**
- **Probleme erkennen, bei denen Stacks, Warteschlangen und Deques geeignete Datenstrukturen sind.**
- den ADT Liste als verlinkte Liste unter Verwendung des Knoten- und Zeigerkonzepts implementieren zu können.
- die Leistung der Implementierung der verknüpften Liste mit der Listenimplementierung von Python vergleichen.


# Was ist eine Queue (Warteschlange)?
Eine Warteschlange ist eine geordnete Sammlung von Artikeln, bei der das Hinzufügen neuer Artikel an einem Ende, dem so genannten "hinteren" Ende, und das Entfernen vorhandener Artikel am anderen Ende, dem so genannten "vorderen" Ende, erfolgt. Wenn ein Element in die Warteschlange eintritt, beginnt es am hinteren Ende und bahnt sich seinen Weg nach vorne, wobei es bis zu dem Zeitpunkt wartet, an dem es das nächste zu entfernende Element ist.

Das zuletzt hinzugefügte Element in der Warteschlange muss am Ende der Sammlung warten. Das Element, das sich am längsten in der Sammlung befindet, steht am Anfang. Dieses Ordnungsprinzip wird manchmal als **FIFO, first-in first-out**, bezeichnet. Es wird auch als "wer zuerst kommt, mahlt zuerst" bezeichnet.

Das einfachste Beispiel für eine Queue ist die typische Warteschlange, an der wir alle von Zeit zu Zeit teilnehmen. Wir warten in einer Schlange auf einen Film, wir warten in der Kassenschlange eines Lebensmittelgeschäfts und wir warten in der Cafeteria-Schlange (damit wir den Tablettstapel aufklappen können). Wohlerzogene Schlangen oder Warteschlangen sind sehr restriktiv, da sie nur einen Weg hinein und nur einen hinaus haben. Es gibt kein Springen in der Mitte und kein Verlassen, bevor man nicht die nötige Zeit gewartet hat, um nach vorne zu kommen. [Abbildung 1](#Queue_Abbildung1) zeigt eine einfache Warteschlange mit Python-Datenobjekten.

<a id='Queue_Abbildung1'></a>
![Abbildung 1](Bilder/ADT/Queue.PNG)
<center><b>Abbildung 1:</b> Eine Warteschlange von Python-Datenobjekten</center>

Auch in der Informatik gibt es gängige Beispiele für Warteschlangen. Unser Computerlabor verfügt über 30 Computer, die mit einem einzigen Drucker vernetzt sind. Wenn die Schülerinnen und Schüler drucken möchten, "stellen sich ihre Druckaufgaben mit allen anderen Druckaufgaben, die warten, in eine Reihe". Die erste Aufgabe ist die nächste, die erledigt werden muss. Wenn Sie als Letzter in der Schlange stehen, müssen Sie warten, bis alle anderen Aufgaben vor Ihnen gedruckt sind. Auf dieses interessante Beispiel werden wir später noch näher eingehen.

Zusätzlich zu den Druckwarteschlangen verwenden Betriebssysteme eine Reihe verschiedener Warteschlangen, um Prozesse innerhalb eines Computers zu steuern. Die Planung dessen, was als Nächstes erledigt wird, basiert in der Regel auf einem Warteschlangenalgorithmus, der versucht, Programme so schnell wie möglich auszuführen und so viele Benutzer wie möglich zu bedienen. Während wir tippen, sind die Tastenanschläge manchmal den Zeichen, die auf dem Bildschirm erscheinen, voraus. Dies ist darauf zurückzuführen, dass der Computer in diesem Moment andere Arbeiten ausführt. Die Tastatureingaben werden in einem schlangenähnlichen Puffer abgelegt, damit sie schließlich in der richtigen Reihenfolge auf dem Bildschirm angezeigt werden können.

# Der abstrakte Datentyp Queue
Der abstrakte Datentyp der Warteschlange wird durch die folgende Struktur und die folgenden Operationen definiert. Eine Warteschlange ist, wie oben beschrieben, als eine geordnete Sammlung von Elementen strukturiert, die an einem Ende, der "Rückseite" genannt, hinzugefügt und am anderen Ende, der "Vorderseite" genannt, entfernt werden. Warteschlangen haben die Eigenschaft einer FIFO-Bestellung. Die Warteschlangenvorgänge sind nachstehend aufgeführt.

+ `Queue()` erstellt eine neue Warteschlange, die leer ist. Sie benötigt keine Parameter und gibt eine leere Warteschlange zurück.
+ `enqueue(item)` fügt ein neues Element am Ende der Warteschlange hinzu. Er benötigt das Element und gibt nichts zurück.
+ `dequeue()` entfernt das vordere Element aus der Warteschlange. Es benötigt keine Parameter und gibt das Element zurück. Die Warteschlange wird modifiziert.
+ `isEmpty()` testet, um festzustellen, ob die Warteschlange leer ist. Er benötigt keine Parameter und gibt einen booleschen Wert zurück.
+ `size()` gibt die Anzahl der Elemente in der Warteschlange zurück. Sie benötigt keine Parameter und gibt eine ganze Zahl zurück.

Wenn wir beispielsweise annehmen, dass `q` eine Warteschlange ist, die erstellt wurde und derzeit leer ist, dann zeigt [Tabelle 1](#Queue_Tabelle1) die Ergebnisse einer Folge von Warteschlangenoperationen. Der Inhalt der Warteschlange ist so dargestellt, dass die Vorderseite rechts ist. 4 war das erste Element, das in die Warteschlange eingereiht wurde, so dass es das erste Element ist, das von der Warteschlange zurückgegeben wird.

<a id='Queue_Tabelle1'></a>
![Tabelle 1](Bilder/ADT/BeispielQueueoperationen.PNG)
<center><b>Tabelle 1:</b> Beispiel für Warteschlangenoperationen</center>  

# Implementierung einer Warteschlange in Python
Es ist wieder angebracht, eine neue Klasse für die Implementierung der Warteschlange für abstrakte Datentypen zu erstellen. Wie zuvor werden wir die Leistungsfähigkeit und Einfachheit der Listensammlung nutzen, um die interne Repräsentation der Warteschlange aufzubauen.

Wir müssen entscheiden, welches Ende der Liste als hinteres und welches als vorderes verwendet werden soll. Die in [Liste 1](#Queue_Liste1) gezeigte Implementierung geht davon aus, dass das hintere Ende an Position 0 in der Liste steht. Dies erlaubt uns, die `insert`-Funktion auf Listen zu verwenden, um neue Elemente am Ende der Warteschlange hinzuzufügen. Die `pop`-Operation kann verwendet werden, um das vordere Element (das letzte Element der Liste) zu entfernen. Erinnern Sie sich, dass dies auch bedeutet, dass Enqueue = O(n) und Dequeue = O(1) sein wird.
<a id='Queue_Liste1'></a>    
**Listing 1**     
```python
class Queue:
    def __init__(self):
        self.items = []
        
    def isEmpty(self):
        return self.items == []
    
    def enqueue(self, item):
        self.items.insert(0,item)
        
    def dequeue(self):
        return self.items.pop()
    
    def size(self):
        return len(self.items)
```
Eine weitere Manipulation dieser Warteschlange würde zu folgenden Ergebnissen führen:     
```python
>>> q.size()
3
>>> q.isEmpty()
False
>>> q.enqueue(8.4)
>>> q.dequeue()
4
>>> q.dequeue()
'dog'
>>> q.size()
2
```
**Codebeispiel 1**

In [5]:
class Queue:
    def __init__(self):
        self.items = []       
    def isEmpty(self):
        return self.items == []    
    def enqueue(self, item):
        self.items.insert(0,item)        
    def dequeue(self):
        return self.items.pop()    
    def size(self):
        return len(self.items)
    
q = Queue()
q.enqueue(4)
q.enqueue('dog')
q.enqueue('True')
print(q.size())
q.isEmpty()
q.enqueue(8.4)
print(q.dequeue())
print(q.dequeue())
print(q.size())

3
4
dog
2


---
### ADT03-01 Selbstest  

In [6]:
hide_me


display(Q1)
display(Q2)

---
# Simulation mit dem ADT Queue: Heiße Kartoffel
Eine der typischen Anwendungen, um eine Warteschlange in Aktion zu zeigen, besteht darin, eine reale Situation zu simulieren, in der Daten nach dem FIFO-Prinzip verwaltet werden müssen. Betrachten wir zunächst das Kinderspiel *Heiße Kartoffel*. Bei diesem Spiel (siehe [Abbildung 2](#Queue_Abbildung2)) stellen sich die Kinder in einem Kreis auf und reichen einen Gegenstand so schnell wie möglich von Nachbar zu Nachbar. An einem bestimmten Punkt im Spiel wird die Aktion gestoppt und das Kind, das den Gegenstand (die Kartoffel) hat, wird aus dem Kreis entfernt. Das Spiel wird so lange fortgesetzt, bis nur noch ein Kind übrig ist.

<a id='Queue_Abbildung2'></a>
![Abbildung 2](Bilder/ADT/heißeKartoffel.PNG)
<center><b>Abbildung 2:</b> Ein Sechs-Personen-Spiel von Heiße Kartoffel</center>

Dieses Spiel ist ein modernes Äquivalent des berühmten Josephus-Problems. Basierend auf einer Legende über den berühmten Historiker Flavius Josephus aus dem ersten Jahrhundert, wird die Geschichte erzählt, dass Josephus und 39 seiner Kameraden bei der jüdischen Revolte gegen Rom in einer Höhle gegen die Römer Widerstand leisteten. Da ihnen eine Niederlage drohte, beschlossen sie, dass sie lieber sterben würden, als Sklaven der Römer zu sein. Sie ordneten sich in einem Kreis an. Ein Mann wurde als Nummer eins bestimmt, und im Uhrzeigersinn töteten sie jeden siebten Mann. Josephus, so die Legende, war unter anderem ein versierter Mathematiker. Er fand sofort heraus, wo er sitzen sollte, um als Letzter zu gehen. Als die Zeit gekommen war, schloss er sich, anstatt sich umzubringen, der römischen Seite an. Sie können viele verschiedene Versionen dieser Geschichte finden. Manche zählen jeden dritten Mann, und manche lassen den letzten Mann auf einem Pferd entkommen. Auf jeden Fall ist die Idee die gleiche.

Wir werden eine allgemeine **Simulation** von Heiße Kartoffel durchführen. Unser Programm wird eine Liste von Namen und eine Konstante, genannt "num", eingeben, die zum Zählen verwendet werden soll. Es wird den Namen der letzten Person zurückgeben, die nach wiederholter Zählung durch `num` übrig bleibt. Was an diesem Punkt geschieht, ist Ihnen überlassen.

Um den Kreis zu simulieren, werden wir eine Warteschlange verwenden (siehe [Abbildung 3](#Queue_Abbildung3)). Nehmen wir an, dass das Kind, das die Kartoffel hält, an der Spitze der Schlange steht. Nach dem Passieren der Kartoffel, wird die Simulation das Kind einfach aus der Warteschlange herausnehmen und es dann sofort wieder in die Warteschlange einreihen, so dass es am Ende der Schlange steht. Es wartet dann, bis alle anderen an der Spitze waren, bevor es wieder an die Reihe kommt. Nach den Operationen `num` dequeue/enqueue wird das Kind an der Spitze permanent entfernt und ein neuer Zyklus beginnt. Dieser Prozess wird so lange fortgesetzt, bis nur noch ein Name übrig bleibt (die Größe der Warteschlange ist 1).

<a id='Queue_Abbildung3'></a>
![Abbildung 3](Bilder/ADT/heißeKartoffelQueue.PNG)
<center><b>Abbildung 3:</b> Eine Warteschlangen-Implementierung von Heiße Kartoffel</center>

Das Programm wird in Listing 2 angezeigt. Ein Aufruf der Funktion `hotPotato` mit 7 als Zählkonstante gibt `Susan` zurück.

**Codebeispiel 1. Hot Potato - Simulation mit dem ADT Queue**

In [7]:
from pythonds.basic import Queue

button = widgets.Button(
    description='Run Code',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)

display(button)

def hotPotato(namelist, num):
    simqueue = Queue()
    for name in namelist:
        simqueue.enqueue(name)

    while simqueue.size() > 1:
        for i in range(num):
            simqueue.enqueue(simqueue.dequeue())

        simqueue.dequeue()

    return simqueue.dequeue()

def button_eventhandler(obj):
    print(hotPotato(["Bill","David","Susan","Jane","Kent","Brad"],7))
    
button.on_click(button_eventhandler)

Button(description='Run Code', style=ButtonStyle())

Beachten Sie, dass in diesem Beispiel der Wert der Zählkonstante größer ist als die Anzahl der Namen in der Liste. Dies ist kein Problem, da sich die Warteschlange wie ein Kreis verhält und die Zählung am Anfang zurückläuft, bis der Wert erreicht ist. Beachten Sie auch, dass die Liste so in die Warteschlange geladen wird, dass der erste Name auf der Liste ganz vorne in der Warteschlange steht. `Bill` ist in diesem Fall der erste Punkt auf der Liste und rückt daher an den Anfang der Warteschlange. Eine Variante dieser Implementierung, die in den Übungen beschrieben wird, ermöglicht einen Zufallszähler.

#### *Bemerkung: GUI-Version des Python-Codes*
Als Alternative zu der klassischen Darstellung mittels verschiedener Ablaufbefehle, wird in diesem Modul beretis eine fortgeschritten Darstellung mit GUI verwendet. Dazu wird eine Schaltfläche ``button = widgets.Button(...)`` definiert, die über einen Event-Handler ``butto_eventhandler(...)``die Simulation startet. Dies erlaubt eine ereignisgesteuerte Kontrolle des Programms und mehrfache Aufrufe durch mehrfaches Klicken des Buttons. 

---
# Simulation: Druck-Aufgaben
Eine weitere interessantere Simulation ermöglicht es uns, das Verhalten der Druckerwarteschlange zu untersuchen, das weiter oben in diesem Abschnitt beschrieben wurde. Erinnern Sie sich daran, dass, wenn Studenten Druckaufgaben an den gemeinsam genutzten Drucker senden, die Aufgaben in eine Warteschlange gestellt werden, um in der Reihenfolge ihres Eingangs bearbeitet zu werden? Bei dieser Konfiguration ergeben sich viele Fragen. Die wichtigste davon könnte sein, ob der Drucker in der Lage ist, eine bestimmte Menge an Arbeit zu bewältigen. Ist dies nicht der Fall, warten die Studenten zu lange auf den Druck und verpassen möglicherweise den nächsten Unterricht (*das Beispiel war noch aus Vor-Corona-Zeiten, wo es Vorlesungen mit physischer Präsenz gab...*).

Betrachten Sie die folgende Situation in einem Informatiklabor. An einem durchschnittlichen Tag arbeiten zu einer bestimmten Stunde etwa 10 Studierende im Labor. Diese Studierenden drucken in der Regel bis zu zweimal in dieser Zeit, und der Umfang dieser Aufgaben reicht von 1 bis 20 Seiten. Der Drucker im Labor ist älter und kann 10 Seiten pro Minute in Entwurfsqualität verarbeiten. Der Drucker könnte umgestellt werden, um eine bessere Qualität zu erzielen, aber dann würde er nur fünf Seiten pro Minute produzieren. Die langsamere Druckgeschwindigkeit könnte die Studenten zu lange warten lassen.  
Welche Seitenrate sollte verwendet werden?

Wir könnten entscheiden, indem wir eine Simulation bauen, die das Labor modelliert. Wir müssen Darstellungen für Studenten, Druckaufgaben und den Drucker erstellen ([Abbildung 4](#Queue_Abbildung4)). Wenn die Studenten Druckaufgaben einreichen, werden wir sie auf eine Warteliste setzen, eine Warteschlange von Druckaufgaben, die an den Drucker angeschlossen ist. Wenn der Drucker eine Aufgabe abgeschlossen hat, schaut er sich die Warteschlange an, um zu sehen, ob noch Aufgaben zu bearbeiten sind. Von Interesse für uns ist die durchschnittliche Zeit, die die Schülerinnen und Schüler auf den Druck ihrer Unterlagen warten werden. Dies entspricht der durchschnittlichen Zeit, die eine Aufgabe in der Warteschlange wartet.

<!--**Abbildung 4:  Druckerwarteschlange des Informatiklabors**-->
<a id='Queue_Abbildung4'></a>
![Abbildung 4](Bilder/ADT/DruckerwarteschlangeInformatiklabor.PNG)
<center><b>Abbildung 4:</b> Druckerwarteschlange des Informatiklabors</center>

Um diese Situation zu modellieren, müssen wir *Wahrscheinlichkeiten* verwenden. Zum Beispiel können Studierende ein Papier von 1 bis 20 Seiten Länge ausdrucken. Wenn jede Länge von 1 bis 20 gleich wahrscheinlich ist, kann die tatsächliche Länge für eine Druckaufgabe simuliert werden, indem eine Zufallszahl zwischen 1 und 20 verwendet wird. Das bedeutet, dass die Wahrscheinlichkeit, dass jede Länge von 1 bis 20 erscheint, gleich groß ist.

Wenn sich 10 Studierende im Labor befinden und jeder zweimal druckt, gibt es durchschnittlich 20 Druckaufgaben pro Stunde. Wie gross ist die Wahrscheinlichkeit, dass zu jeder Sekunde eine Druckaufgabe erstellt wird? Diese Frage lässt sich beantworten, indem man das Verhältnis von Aufgaben und Zeit berücksichtigt. Zwanzig Aufgaben pro Stunde bedeutet, dass im Durchschnitt *alle 180 Sekunden eine Aufgabe* erstellt wird:

$$\frac{20 Aufgaben}{1 Stunde} * \frac{1 Stunde}{60 Minuten} * \frac{1 Minute}{60 Sekunden} = \frac{1 Aufgabe}{180 Sekunden}$$

Für jede Sekunde können wir die Wahrscheinlichkeit simulieren, dass ein Druckauftrag stattfindet, indem wir eine Zufallszahl zwischen 1 und 180 einschließlich generieren. Wenn die Zahl 180 ist, sagen wir, dass eine Aufgabe erstellt wurde. Beachten Sie, dass es möglich ist, dass viele Aufgaben in einer Reihe erstellt werden, oder wir können eine ganze Weile warten, bis eine Aufgabe erscheint. Das liegt in der Natur der Simulation. Sie möchten die reale Situation so genau wie möglich simulieren, da Sie die allgemeinen Parameter kennen.

## Haupt-Simulationsschritte
Hier ist die Hauptsimulation.    
1. Erstellen Sie eine Warteschlange von Druckaufträgen. Jede Aufgabe wird bei ihrer Ankunft mit einem Zeitstempel versehen. Die Warteschlange ist zu Beginn leer.
2. Für jede Sekunde (`currentSecond`):
    + Wird ein neuer Druckauftrag erstellt? Wenn ja, fügen Sie ihn mit `currentSecond` als Zeitstempel in die Warteschlange ein.
    + Wenn der Drucker nicht besetzt ist und wenn eine Aufgabe wartet:
        - Entfernen Sie die nächste Aufgabe aus der Druckerwarteschlange und weisen Sie sie dem Drucker zu.
        - Subtrahieren Sie den Zeitstempel von `currentSecond`, um die Wartezeit für diese Aufgabe zu berechnen.
         - Hängen Sie die Wartezeit für diese Aufgabe zur späteren Verarbeitung an eine Liste an.
         - Finden Sie anhand der Anzahl der Seiten des Druckauftrags heraus, wie viel Zeit erforderlich sein wird.
    + Der Drucker druckt nun eine Sekunde lang, falls erforderlich. Er zieht auch eine Sekunde von der für diese Aufgabe benötigten Zeit ab.
    + Wenn die Aufgabe abgeschlossen ist, d.h. die benötigte Zeit Null erreicht hat, ist der Drucker nicht mehr beschäftigt.
3. Nachdem die Simulation abgeschlossen ist, berechnen Sie die durchschnittliche Wartezeit aus der Liste der erzeugten Wartezeiten.

##  Implementierung in Python
Um diese Simulation zu entwerfen, werden wir Klassen für die drei oben beschriebenen Objekte der realen Welt erstellen: `Printer`, `Task` und `PrintQueue`.

Die `Printer` Klasse ([Listing 3](#Queue_Liste2)) muss verfolgen, ob sie eine aktuelle Aufgabe hat. Ist dies der Fall, dann ist sie beschäftigt (Zeilen 13-17), und die benötigte Zeit kann anhand der Anzahl der Seiten der Aufgabe berechnet werden. Der Konstruktor ermöglicht auch die Initialisierung der Einstellung Seiten pro Minute. Die `tick` Methode dekrementiert den internen Zeitgeber und setzt den Drucker auf Leerlauf (Zeile 11), wenn die Aufgabe abgeschlossen ist.

<a id='Queue_Liste3'></a>
**Listing 3**
```python
class Printer:
    def __init__(self, ppm):
        self.pagerate = ppm
        self.currentTask = None
        self.timeRemaining = 0

    def tick(self):
        if self.currentTask != None:
            self.timeRemaining = self.timeRemaining - 1
            if self.timeRemaining <= 0:
                self.currentTask = None

    def busy(self):
        if self.currentTask != None:
            return True
        else:
            return False

    def startNext(self,newtask):
        self.currentTask = newtask
        self.timeRemaining = newtask.getPages() * 60/self.pagerate
```

Die Aufgabenklasse ([Listing 4](#Queue_Liste4)) wird eine einzelne Druckaufgabe darstellen. Wenn die Aufgabe erstellt wird, liefert ein Zufallszahlengenerator eine Länge von 1 bis 20 Seiten. Wir haben uns für die Verwendung der Funnktion `randrange` aus dem Modul `random` entschieden.

```python
>>> import random
>>> random.randrange(1,21)
18
>>> random.randrange(1,21)
8
>>>
```

Jede Aufgabe muss außerdem einen Zeitstempel beinhalten, der für die Berechnung der Wartezeit verwendet wird. Dieser Zeitstempel stellt die Zeit dar, zu der der Task erstellt und in die Druckerwarteschlange gestellt wurde. Die Methode `waitTime` kann dann verwendet werden, um die in der Warteschlange verbrachte Zeit vor dem Druckbeginn abzurufen.

<a id='Queue_Liste4'></a>
**Listing 4. Klasse der Tasks**
```python
import random

class Task:
    def __init__(self,time):
        self.timestamp = time
        self.pages = random.randrange(1,21)

    def getStamp(self):
        return self.timestamp

    def getPages(self):
        return self.pages

    def waitTime(self, currenttime):
        return currenttime - self.timestamp
```

Die Hauptsimulation ([Listing 5](#Queue_Liste4)) implementiert den oben beschriebenen Algorithmus. Das Objekt `printQueue` ist eine Instanz unserer bestehenden Warteschlange ADT. Eine boolesche Hilfsfunktion, `newPrintTask`, entscheidet, ob ein neuer Druckauftrag erstellt wurde. Wir haben uns erneut dafür entschieden, die Funktion `randrange` aus dem Modul `random` zu verwenden, um eine zufällige ganze Zahl zwischen 1 und 180 zurückzugeben. Druckaufgaben kommen alle 180 Sekunden einmal an. Durch die willkürliche Auswahl von 180 aus dem Bereich der Zufallsganzzahlen (Zeile 32) können wir dieses Zufallsereignis simulieren. Die Simulationsfunktion ermöglicht es uns, die Gesamtzeit und die Seiten pro Minute für den Drucker festzulegen.

<a id='Queue_Liste4'></a>
**Listing 5. Hauptsimulation**
```python
from pythonds.basic.queue Queue

import random

def simulation(numSeconds, pagesPerMinute):
    labprinter = Printer(pagesPerMinute)
    printQueue = Queue()
    waitingtimes = []

    for currentSecond in range(numSeconds):
      if newPrintTask():
         task = Task(currentSecond)
         printQueue.enqueue(task)

      if (not labprinter.busy()) and (not printQueue.isEmpty()):
        nexttask = printQueue.dequeue()
        waitingtimes.append(nexttask.waitTime(currentSecond))
        labprinter.startNext(nexttask)

      labprinter.tick()

    averageWait=sum(waitingtimes)/len(waitingtimes)
    print("Average Wait %6.2f secs %3d tasks remaining."%(averageWait,printQueue.size()))

def newPrintTask():
    num = random.randrange(1,181)
    if num == 180:
        return True
    else:
        return False

for i in range(10):
    simulation(3600,5)
```` 

Wenn wir die Simulation durchführen, sollten wir uns keine Sorgen machen, dass die Ergebnisse jedes Mal anders ausfallen. Dies ist auf die probabilistische Natur der Zufallszahlen zurückzuführen. Wir sind an den Trends interessiert, die bei der Anpassung der Simulationsparameter auftreten können. Hier sind einige Ergebnisse.

Zunächst werden wir die Simulation über einen Zeitraum von 60 Minuten (3.600 Sekunden) mit einer Seitenzahl von fünf Seiten pro Minute durchführen. Darüber hinaus werden wir 10 unabhängige Versuche durchführen. Denken Sie daran, dass, da die Simulation mit Zufallszahlen arbeitet, jeder Durchlauf unterschiedliche Ergebnisse liefert.

```python
>>>for i in range(10):
      simulation(3600,5)

Average Wait 165.38 secs 2 tasks remaining.
Average Wait  95.07 secs 1 tasks remaining.
Average Wait  65.05 secs 2 tasks remaining.
Average Wait  99.74 secs 1 tasks remaining.
Average Wait  17.27 secs 0 tasks remaining.
Average Wait 239.61 secs 5 tasks remaining.
Average Wait  75.11 secs 1 tasks remaining.
Average Wait  48.33 secs 0 tasks remaining.
Average Wait  39.31 secs 3 tasks remaining.
Average Wait 376.05 secs 1 tasks remaining.
```

Nachdem wir unsere 10 Versuche durchgeführt haben, können wir feststellen, dass die mittlere durchschnittliche Wartezeit 122,09 Sekunden beträgt. Sie können auch sehen, dass es eine große Variation in der durchschnittlichen Gewichtszeit mit einem Minimum von durchschnittlich 17,27 Sekunden und einem Maximum von 376,05 Sekunden gibt. Sie können auch feststellen, dass in nur zwei der Fälle alle Aufgaben erledigt wurden.

Nun werden wir die Seitenzahl auf 10 Seiten pro Minute anpassen und die 10 Versuche erneut durchführen, wobei wir hoffen, dass mit einer schnelleren Seitenzahl mehr Aufgaben in dem Zeitrahmen von einer Stunde erledigt werden können.

```python
>>>for i in range(10):
      simulation(3600,10)

Average Wait   1.29 secs 0 tasks remaining.
Average Wait   7.00 secs 0 tasks remaining.
Average Wait  28.96 secs 1 tasks remaining.
Average Wait  13.55 secs 0 tasks remaining.
Average Wait  12.67 secs 0 tasks remaining.
Average Wait   6.46 secs 0 tasks remaining.
Average Wait  22.33 secs 0 tasks remaining.
Average Wait  12.39 secs 0 tasks remaining.
Average Wait   7.27 secs 0 tasks remaining.
Average Wait  18.17 secs 0 tasks remaining.
```

Sie können die Simulation für sich selbst in **Codebeispiel 2** ausführen.

**Codebeispiel 2. Simulation der Druckeraufträge**

In [9]:
from pythonds.basic import Queue
import random

sekunden= widgets.Text(
    value='',
    placeholder='Zeitspanne',
    disabled=False
)

seiten= widgets.Text(
    value='',
    placeholder='Seiten pro Minute',
    disabled=False
)

button = widgets.Button(
    description='Run Code',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)

display(sekunden, seiten, button)

class Printer:
    def __init__(self, ppm):
        self.pagerate = ppm
        self.currentTask = None
        self.timeRemaining = 0

    def tick(self):
        if self.currentTask != None:
            self.timeRemaining = self.timeRemaining - 1
            if self.timeRemaining <= 0:
                self.currentTask = None

    def busy(self):
        if self.currentTask != None:
            return True
        else:
            return False

    def startNext(self,newtask):
        self.currentTask = newtask
        self.timeRemaining = newtask.getPages() * 60/self.pagerate

class Task:
    def __init__(self,time):
        self.timestamp = time
        self.pages = random.randrange(1,21)

    def getStamp(self):
        return self.timestamp

    def getPages(self):
        return self.pages

    def waitTime(self, currenttime):
        return currenttime - self.timestamp


def simulation(numSeconds, pagesPerMinute):

    labprinter = Printer(pagesPerMinute)
    printQueue = Queue()
    waitingtimes = []

    for currentSecond in range(numSeconds):

      if newPrintTask():
         task = Task(currentSecond)
         printQueue.enqueue(task)

      if (not labprinter.busy()) and (not printQueue.isEmpty()):
        nexttask = printQueue.dequeue()
        waitingtimes.append( nexttask.waitTime(currentSecond))
        labprinter.startNext(nexttask)

      labprinter.tick()

    averageWait=sum(waitingtimes)/len(waitingtimes)
    print("Average Wait %6.2f secs %3d tasks remaining."%(averageWait,printQueue.size()))

def newPrintTask():
    num = random.randrange(1,181)
    if num == 180:
        return True
    else:
        return False

#for i in range(10):
    #simulation(3600,5)
    
def button_eventhandler(obj):
    for i in range(10):
        simulation(3600,5)
    
button.on_click(button_eventhandler)

Text(value='', placeholder='Zeitspanne')

Text(value='', placeholder='Seiten pro Minute')

Button(description='Run Code', style=ButtonStyle())

## Diskussion
Wir haben versucht, die Frage zu beantworten, ob der aktuelle Drucker die Last der Aufgabe bewältigen könnte, wenn er so eingestellt wäre, dass er mit einer besseren Qualität, aber einer langsameren Seitenrate druckt. Der Ansatz, den wir verfolgt haben, bestand darin, eine Simulation zu schreiben, die die Druckaufgaben als Zufallsereignisse unterschiedlicher Länge und Ankunftszeiten modelliert.

Die obige Ausgabe zeigt, dass bei einem Druck von 5 Seiten pro Minute die durchschnittliche Wartezeit von einem Tiefstwert von 17 Sekunden bis zu einem Höchstwert von 376 Sekunden (etwa 6 Minuten) variierte. Bei einer schnelleren Druckgeschwindigkeit betrug der niedrige Wert 1 Sekunde bei einem Hoch von nur 28. Hinzu kam, dass in 8 von 10 Läufen mit 5 Seiten pro Minute am Ende der Stunde noch Druckaufträge in der Warteschlange standen.

Daher kann gesagt werden, dass eine Verlangsamung des Druckers, um eine bessere Qualität zu erhalten, vielleicht keine gute Idee ist. Die Studenten können es sich nicht leisten, so lange auf ihre Arbeiten zu warten, besonders dann nicht, wenn sie in die nächste Klasse gehen müssen. Eine sechsminütige Wartezeit wäre einfach zu lang.

Diese Art der Simulationsanalyse ermöglicht es uns, viele Fragen zu beantworten, die allgemein als "Was-wäre-wenn"-Fragen bekannt sind. Wir brauchen nur die von der Simulation verwendeten Parameter zu variieren, und schon können wir eine beliebige Anzahl interessanter Verhaltensweisen simulieren, zum Beispiel:
+ Was passiert, wenn die Einschreibungen steigen und die durchschnittliche Zahl der Studierenden um 20 zunimmt?
+ Was ist, wenn es Samstag ist und die Studenten nicht zum Unterricht kommen müssen? Können sie es sich leisten, zu warten?
+ Was ist, wenn die Größe des durchschnittlichen Druckauftrags abnimmt, da Python eine so mächtige Sprache ist und Programme dazu neigen, viel kürzer zu sein?

Diese Fragen könnten alle durch eine Modifikation der obigen Simulation beantwortet werden. Es ist jedoch wichtig, daran zu denken, dass die Simulation nur so gut ist wie die Annahmen, die zu ihrer Erstellung verwendet werden. Reale Daten über die Anzahl der Druckaufgaben pro Stunde und die Anzahl der Studenten pro Stunde waren notwendig, um eine robuste Simulation zu erstellen.

---
### ADT03-01 Praktischer Selbstest 
Wie würden Sie die Druckersimulation modifizieren, um *eine größere Anzahl von Studierenden* abzubilden? Angenommen, die Anzahl der Studenten würde sich *verdoppeln*. Sie müssen einige vernünftige Annahmen darüber treffen, wie diese Simulation zusammengesetzt wurde, aber was würden Sie ändern? Ändern Sie den Code. Nehmen Sie auch an, dass die *Länge der durchschnittlichen Druckaufgabe halbiert* würde. Ändern Sie den Code, um dieser Änderung Rechnung zu tragen. Wie würden Sie schließlich die *Anzahl der Studenten parametrisieren*? Anstatt den Code zu ändern, möchten wir die Anzahl der Studenten zu einem Parameter der Simulation machen.

---
#  Der abstrakte Datentyp *Deque*?
Eine **Deque**, auch bekannt als **doppelseitige Warteschlange**, ist eine geordnete Sammlung von Elementen, die der Warteschlange ähnlich ist. Sie hat zwei Enden, ein vorderes und ein hinteres, und die Gegenstände bleiben in der Sammlung positioniert. Was eine Deque unterscheidet, ist die uneingeschränkte Art des Hinzufügens und Entfernens von Gegenständen. Neue Artikel können entweder an der Vorderseite oder an der Rückseite hinzugefügt werden. Ebenso können bestehende Objekte an beiden Enden entfernt werden. In gewisser Weise bietet diese hybride lineare Struktur alle Möglichkeiten von Stapeln und Warteschlangen in einer einzigen Datenstruktur. [Abbildung 5](#Deque_Abbildung1) zeigt eine Übersicht über Python-Datenobjekte.

Es ist wichtig zu beachten, dass die Deque, obwohl sie viele der Merkmale von Stacks und Warteschlangen annehmen kann, nicht die LIFO- und FIFO-Ordnungen benötigt, die durch diese Datenstrukturen erzwungen werden. Es liegt am Entwickler, die Einfügen- und Löschoperationen einheitlich und konsistent zu nutzen.

<!--**Abbildung 5. Deque von Python-Datenobjekten**-->
<a id='Deque_Abbildung1'></a>
![Abbildung 5](Bilder/ADT/Deque.PNG)
<center><b>Abbildung 5:</b> Eine Deque von Python-Datenobjekten</center>

# Der abstrakte Datentyp Deque
Der abstrakte Datentyp Deque wird durch die folgende Struktur und Operationen definiert. Eine Deque ist, wie oben beschrieben, als eine geordnete Sammlung von Elementen strukturiert, bei der Elemente an beiden Enden, entweder vorne oder hinten, hinzugefügt und entfernt werden. Die Deque-Operationen sind nachstehend aufgeführt.
+ `Deque()` schafft eine neue Deque, die leer ist. Sie benötigt keine Parameter und gibt eine leere Deque zurück.
+ `addFront(item)` fügt einen neuen Gegenstand auf der Vorderseite der Deque hinzu. Er benötigt den Gegenstand und gibt nichts zurück.
+ `addRear(item)` fügt ein neues Element auf der Rückseite der Deque hinzu. Er benötigt das Element und gibt nichts zurück.
+ `removeFront()` entfernt das vordere Element aus der Deque. Es benötigt keine Parameter und gibt das Element zurück. Die Deque wird geändert.
+ `removeRear()` entfernt das hintere Element aus der Deque. Er benötigt keine Parameter und gibt das Element zurück. Die Deque wird geändert.
+ `isEmpty()` testet, ob die Deque leer ist. Er benötigt keine Parameter und gibt einen booleschen Wert zurück.
+ `size()` gibt die Anzahl der Elemente in der Deque zurück. Sie benötigt keine Parameter und gibt eine ganze Zahl zurück.

Wenn wir zum Beispiel annehmen, dass `d` eine Deque ist, die erstellt wurde und derzeit leer ist, dann zeigt [Tabelle 2](#Deque_Tabelle2) die Ergebnisse einer Sequenz von Dequeoperationen. Beachten Sie, dass die vorangestellten Inhalte auf der rechten Seite aufgelistet sind. Es ist sehr wichtig, den Überblick über die Vorder- und Rückseite zu behalten, wenn Sie Elemente in die Sammlung hinein und aus ihr heraus verschieben, da die Dinge etwas verwirrend sein können.

<!--**Tabelle 2: Beispiele für Deque-Operationen**-->
<a id='Deque_Tabelle2'></a>
![Tabelle 2](Bilder/ADT/BeispielDequeoperationen.PNG)
<center><b>Tabelle 2:</b> Beispiele für Deque-Operationen</center>

# Deque in Python implementieren
Wie wir es in den vorherigen Abschnitten getan haben, werden wir eine neue Klasse für die Implementierung des abstrakten Datentyps `deque` erstellen. Auch hier wird die Python-Liste einen Satz von Methoden bieten, auf denen die Details des deque aufgebaut werden können. Unsere Implementierung ([Liste 1](#Deque_Liste1)) geht davon aus, dass sich die Rückseite des Deque an Position 0 in der Liste befindet.

<a id='Deque_Liste1'></a>
**Listing 6**  
```python
class Deque:
    def __init__(self):
        self.items = []

    def isEmpty(self):
        return self.items == []

    def addFront(self, item):
        self.items.append(item)

    def addRear(self, item):
        self.items.insert(0,item)

    def removeFront(self):
        return self.items.pop()

    def removeRear(self):
        return self.items.pop(0)

    def size(self):
        return len(self.items)
```

In `removeFront` benutzen wir die `pop`-Methode um das letze Element der Liste zu entfernen. In `removeRear` muss die Methode `pop(0)` jedoch das erste Element der Liste entfernen. Ebenso müssen wir die `insert`-Methode (Zeile 12) in `addRear` verwenden, da die `append`-Methode das Hinzufügen eines neuen Elements am Ende der Liste voraussetzt.

Sie können viele Ähnlichkeiten mit dem bereits beschriebenen Python-Code für Stacks und Warteschlangen erkennen. Sie werden wahrscheinlich auch feststellen, dass in dieser Implementierung das Hinzufügen und Entfernen von Elementen von vorne O(1) ist, während das Hinzufügen und Entfernen von hinten O(n) ist. Dies ist angesichts der üblichen Operationen, die beim Hinzufügen und Entfernen von Elementen auftreten, zu erwarten. Auch hier ist es wichtig sicherzustellen, dass wir wissen, wo die Vorder- und Rückseite in der Implementierung zugewiesen sind.

# Beispiel Deque: ein Palindrome-Checker
Ein interessantes Problem, das mit Hilfe der Datenstruktur Deque leicht gelöst werden kann, ist das klassische Palindrom-Problem. Ein **Palindrom** ist eine Zeichenfolge, die vorwärts und rückwärts dasselbe liest, z.B. *Radar*, *Toot* und *Madam*. Wir möchten einen Algorithmus zur Eingabe einer Zeichenfolge konstruieren und prüfen, ob es sich um ein Palindrom handelt.

Die Lösung dieses Problems besteht darin, die Zeichen der Zeichenfolge in einer Deque zu speichern. Wir werden die Zeichenfolge von links nach rechts verarbeiten und jedes Zeichen auf der Rückseite der Deque hinzufügen. Zu diesem Zeitpunkt verhält sich die Deque sehr ähnlich wie eine gewöhnliche Warteschlange. Jetzt können wir jedoch die doppelte Funktionalität der Deque nutzen. Die Vorderseite der Deque enthält das erste Zeichen der Zeichenkette und die Rückseite der Deque das letzte Zeichen (siehe [Abbildung 6](#Deque_Abbildung6)). 

<!--**Abb. 6. Palindrome als ADT Dequeue**-->
<a id='Deque_Abbildung6'></a>
![Abbildung 2](Bilder/ADT/DequePalindrom.PNG)
<center><b>Abbildung 6:</b> Eine Deque</center>

Da wir beide direkt entfernen können, können wir sie vergleichen und nur dann fortfahren, wenn sie übereinstimmen. Wenn wir die erste und die letzte Übereinstimmung beibehalten können, gehen uns irgendwann entweder die Zeichen aus oder wir erhalten eine Deque der Größe 1, je nachdem, ob die Länge der ursprünglichen Zeichenfolge gerade oder ungerade war. In beiden Fällen muss die Zeichenfolge ein Palindrom sein. Die vollständige Funktion zur Palindrom-Prüfung erscheint in Codebeispiel 3.

**Codebeispiel 3. Palindrome-Checker mit ADT Dequeue**

In [8]:
from pythonds.basic import Deque

print("Achten Sie auf die Groß- und Kleinschreibung. A und a sind in diesem Programm zwei unterschiedliche Buchstaben")

eingabe= widgets.Text(
    value='',
    placeholder='Wort einfügen',
    disabled=False
)

button = widgets.Button(
    description='Run Code',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)

display(eingabe, button)

def palchecker(aString):
    chardeque = Deque()

    for ch in aString:
        chardeque.addRear(ch)

    stillEqual = True

    while chardeque.size() > 1 and stillEqual:
        first = chardeque.removeFront()
        last = chardeque.removeRear()
        if first != last:
            stillEqual = False

    return stillEqual

def button_eventhandler(obj):
    print(palchecker(eingabe.value))
    
button.on_click(button_eventhandler)

Achten Sie auf die Groß- und Kleinschreibung. A und a sind in diesem Programm zwei unterschiedliche Buchstaben


Text(value='', placeholder='Wort einfügen')

Button(description='Run Code', style=ButtonStyle())